# Inżynieria lingwistyczna
Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

# Zadanie domowe 2

In [4]:
import pandas as pd
import numpy as np
       

## Zadanie 1 - eksploracja przestrzeni zagnieżdżeń
Wczytajmy do przestrzeni plik zagnieżdżeń, który należy pobrać ze strony:
https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.pl.vec Są to zagnieżdzenia dla języka polskiego uzyskane systemem fastText.

Do przestrzeni wczytujemy tylko 100 tys. najczęstrzych słów, tak aby operacje przebiegały szybciej.

In [5]:
import io
def load_vectors(fname, limit = 100000):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    n = min(n,limit)
    embeddings = np.empty((n,d), dtype = np.float)
    words_idx = []
    for i, line in enumerate(fin):
        if i >= limit:
            break
        tokens = line.rstrip().split(' ')
        words_idx.append(tokens[0])
        embeddings[i] =  np.array(tokens[1:]).astype(np.float)
    return words_idx, embeddings
words_idx, embeddings = load_vectors('wiki.pl.vec')

Poniższe zadania mają na celu poekserymentowanie z przestrzenią zagnieżdżeń, ale też zrozumienie stojącymi za nich operacji. Dozwolone jest korzystanie tylko z podstawowych operatorów Python i numpy (w szczególności zakaz dotyczy: sklearn, gensim, fasttext itd.)

Jeśli potrzebujesz do dalszego przetwarzania przeprowadzenia jakichś normalizacji macierzy -- możesz wstępnie przetworzyć macierz zagnieżdzeń poniżej. Pamiętaj, że sprawdzarka będzie używała wywołań do `embeddings` (i `words_idx`) -- musisz nadpisać macierz zagnieżdzeń. To pole jest pomocnicze i nie podlega ocenie.

In [37]:
# YOUR CODE HERE
words_idx = dict([(w, i) for i, w in enumerate(words_idx)])
words_idx

{'</s>': 0,
 '.': 1,
 ',': 2,
 'w': 3,
 '-': 4,
 ')': 5,
 '(': 6,
 "'": 7,
 'i': 8,
 '–': 9,
 'na': 10,
 'z': 11,
 'do': 12,
 'się': 13,
 'align': 14,
 'roku': 15,
 'a': 16,
 'jest': 17,
 'rd': 18,
 'nie': 19,
 'od': 20,
 'o': 21,
 'to': 22,
 '#': 23,
 'przez': 24,
 'left': 25,
 'po': 26,
 'score': 27,
 'jako': 28,
 'oraz': 29,
 '"': 30,
 'że': 31,
 'został': 32,
 'był': 33,
 'm': 34,
 'ur': 35,
 'za': 36,
 'r': 37,
 'jego': 38,
 'dla': 39,
 'center': 40,
 'tym': 41,
 'pod': 42,
 'cest': 43,
 'latach': 44,
 'dyskusja': 45,
 'są': 46,
 'jak': 47,
 'the': 48,
 'też': 49,
 '/': 50,
 'ze': 51,
 'miejsce': 52,
 'polski': 53,
 'linki': 54,
 'zewnętrzne': 55,
 'co': 56,
 'także': 57,
 'lub': 58,
 'tego': 59,
 '?': 60,
 'right': 61,
 'in': 62,
 'przy': 63,
 'cet': 64,
 'team': 65,
 'ale': 66,
 'który': 67,
 's': 68,
 'jednak': 69,
 'ii': 70,
 'ma': 71,
 'również': 72,
 'jej': 73,
 'de': 74,
 'lat': 75,
 'brak': 76,
 'we': 77,
 'of': 78,
 'seed': 79,
 'podczas': 80,
 'była': 81,
 'które': 82,
 

Zaimplementuj funkcję, która obliczy podobieństwo kosinusowe pomiędzy dwoma wyrazami.

In [10]:
def calc_sim(word_a, word_b, words_idx, embeddings):
    # YOUR CODE HERE
    # from: https://skipperkongen.dk/2018/09/19/cosine-similarity-in-python/
    a = embeddings[words_idx[word_a]]
    b = embeddings[words_idx[word_b]]
    dot = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    return dot / (norm_a * norm_b)

In [11]:
from nose.tools import assert_almost_equal
assert_almost_equal(calc_sim("bieber", "rihanna", words_idx, embeddings), calc_sim("rihanna", "bieber", words_idx, embeddings))

Policz podobieństwo pomiędzy wyrazem `bieber` a wyrazami:
    - `rihanna`
    - `piłsudski`
    - `kanada`
    - `polska`
    - `piosenka`

In [12]:
calc_sim("bieber", "rihanna", words_idx, embeddings)

0.524583248263655

In [15]:
# YOUR CODE HERE
for row in ['rihanna', 'piłsudski', 'kanada', 'polska', 'piosenka']:
    print(row, calc_sim('bieber', row, words_idx, embeddings))

rihanna 0.524583248263655
piłsudski 0.1930395805146356
kanada 0.20042742126487934
polska 0.12505934735679372
piosenka 0.2874871368858332


Zaimplementuj funkcję, która zwróci najbardziej podobne słowa (miara kosinusowa) do danego słowa `word`. W wyniku wypisz tylko `top` słów z najbliższymi zagnieżdżeniami, pomijając słowo `word`.

In [58]:
import heapq

def find_similar(word, words_idx, embedding_matrix, top=10):
    # YOUR CODE HERE
    results = []
    for key in words_idx.keys():
        sim = calc_sim(word, key, words_idx, embedding_matrix)
        result = (sim, key)
        
        if len(results) <= top + 1:
            heapq.heappush(results, result)
        else:
            heapq.heapreplace(results, result)
            
#     print([i[1] for i in heapq.nlargest(50, results)])
    similarities = []
    for result in heapq.nlargest(top + 1, results):
        
        similar_word = result[1]
        if similar_word != word:
            similarities.append(similar_word)
    
    return similarities


In [59]:
assert len(find_similar("radość", words_idx, embeddings)) == 10

Znajdź najbardziej podobne słowa do kobieta, politechnika, mateusz, szczecin, niemcy, piłsudski

In [60]:
find_similar("kobieta", words_idx, embeddings)


['kobietą',
 'dziewczyna',
 'mężczyzna',
 'kobietę',
 'dziewczynka',
 'mężczyznę',
 'staruszka',
 'mężczyzną',
 'kobiecie',
 'mężczyzny']

In [61]:
# YOUR CODE HERE
for row in ['kobieta', 'politechnika', 'mateusz', 'szczecin', 'niemcy', 'piłsudski']:
    print(find_similar(row, words_idx, embeddings))

['kobietą', 'dziewczyna', 'mężczyzna', 'kobietę', 'dziewczynka', 'mężczyznę', 'staruszka', 'mężczyzną', 'kobiecie', 'mężczyzny']
['politechniki', 'politechniką', 'politechnikę', 'politechniczny', 'politechnice', 'politechnicznej', 'politechnicznego', 'politechnicznym', 'inżynierska', 'elektrotechnika']
['łukasz', 'bartłomiej', 'bartosz', 'kacper', 'marcin', 'mateusza', 'tomasz', 'patryk', 'rafał', 'mateuszem']
['szczecinek', 'szczeciński', 'szczecinem', 'gryfino', 'szczecinie', 'stargard', 'szczecina', 'koszalin', 'szczecińska', 'świnoujście']
['niemieccy', 'naziści', 'alianci', 'okupanci', 'polacy', 'hitlerowcy', 'niemieckie', 'rosjanie', 'niemców', 'niemcom']
['piłsudskim', 'piłsudskiego', 'piłsudskiemu', 'sosnkowski', 'mościcki', 'śmigły', 'józef', 'żeligowski', 'piłsudczyków', 'sosnkowskiego']


Krótko skomentuj wyniki dla słowa `niemcy`. Które z powstałych analogii biorą się z semantycznego powiązania a które z semantycznego podobieństwa?

In [64]:
# YOUR CODE HERE
# Zabarwienie historyczne - powiązania z III Rzeszą i elementami wojennymi
#
# powiązanie: "naziści", "alianci", "okupanci", "polacy", "hitlerowcy", "rosjanie"
# podobieństwo: "niemieccy", "niemieckie", "niemców", "niemcom"

Zaimplementuj funkcje szukającą brakującego elementu relacji ,,`word_a` jest do `word_a2` jak `word_b` jest do...''. Funkcja powinna zwrócić 10 najbardziej pasujących słow z pominięciem słów będących jej argumentami.

In [65]:
def find_similar_pair(word_a, word_a2, word_b,  words_idx, matrix, top=10):
    # YOUR CODE HERE
    
    excl = [word_a, word_a2, word_b]
    
    results = []
    for key in words_idx.keys():
        sim = calc_sim(key, word_b, words_idx, matrix) - calc_sim(key, word_a, words_idx, matrix) + calc_sim(key, word_a2, words_idx, matrix)
        result = (sim, key)
        
        if len(results) <= top + 1:
            heapq.heappush(results, result)
        else:
            heapq.heapreplace(results, result)
            
#     print([i[1] for i in heapq.nlargest(50, results)])
    similarities = []
    for result in heapq.nlargest(top + 1, results):
        
        similar_word = result[1]
        if similar_word not in excl:
            similarities.append(similar_word)
    
    return similarities


In [66]:
assert find_similar_pair( "mężczyzna", "król", "kobieta", words_idx, embeddings)[0] == "królowa"

Pieniądze są do profesora jak wiedza do...

In [67]:
# YOUR CODE HERE
find_similar_pair("pieniądze", "profesor", "wiedza", words_idx, embeddings)

['habilitowany',
 'docent',
 'wykładowca',
 'profesorem',
 'habilitacja',
 'adiunkt',
 'rektor',
 'profesora',
 'humanistycznych']

Mateusza jest do mateusz jak łukasza do ...

In [68]:
# YOUR CODE HERE
find_similar_pair("mateusza", "mateusz", "łukasza", words_idx, embeddings)

['łukasz',
 'bartłomiej',
 'bartosz',
 'maciej',
 'tomasz',
 'rafał',
 'patryk',
 'marcin',
 'michał']

Warszawa jest do "polska" jak "berlin" do ...

In [69]:
# YOUR CODE HERE
find_similar_pair("warszawa", "polska", "berlin", words_idx, embeddings)

['niemiecka',
 'berliner',
 'wschodnioniemiecka',
 'berlińska',
 'deutschland',
 'deutsche',
 'brandenburg',
 'germany',
 'berlinem']

Zurich jest do ETH jak Poznań do ...

In [70]:
find_similar_pair( "zurich", "eth", "poznań", words_idx, embeddings)

['„poznań',
 'wrocław',
 'poznania',
 'poznańskie',
 'gniezno',
 'kraków',
 'poznaniu',
 'uam',
 'wielkopolski']

Niemcy są do Merkel jak Polska do ...

In [71]:
find_similar_pair( "niemcy", "merkel", "polska", words_idx, embeddings)

['kaczyńska',
 'ekonomistka',
 'lewandowska',
 'kwaśniewska',
 'lekarka',
 'parlamentarzystka',
 'marcinkiewicz',
 'nowacka',
 'dziennikarka']

Na wektorach możemy wykonywać standardowe operacje algebry liniowej takie jak np. projekcja czyli rzutowanie danych na jakichś zbiór osi (więcej: notatki z algebry liniowej np. https://ocw.mit.edu/courses/mathematics/18-06sc-linear-algebra-fall-2011/least-squares-determinants-and-eigenvalues/projections-onto-subspaces/). W szczególności może to się przydać do zrzutowania słowa na przestrzeń w której pewny wybrany kierunek (wskazywany przez wektor) jest eliminowany.

Do czego może to się przydać? Jeśli uruchomisz funkcję `find_similar` dla słowa ,,mateusza'' znajdziesz m.in. ,,łukasza'' ale także ,,ewangelia'', ,,ewangelisty'' i ,,apostoła''. Chcąc pominąc kontekst religijny tego słowa możesz zrzutować jego reprezentacje na przestrzeń bez wektora ,,ewangelia'' i poszukać jego najbliższych sąsiadów (którymi będą teraz po prostu imiona męskie). Zaimplementuj taką funkcję.


In [72]:
def find_similar_with_rejection(word, remove, words_idx, matrix, top=10):
    """
    Działanie analogiczne do find_similar z dodatkowym parametrem remove, 
    który jest *listą* słów, które należy wyrzucić poprzez projekcję.
    Dla remove=[] powinno się zwracać dokładnie to samo co find_similar
    """
    # YOUR CODE HERE
    if not remove:
        return find_similar(word, words_idx, matrix, top)
    else:
        new_matrix = matrix.copy()
        index = words_idx[word]
        
        for w in remove:
            new_matrix[index] = new_matrix[index] - new_matrix[words_idx[w]]
        
    return find_similar(word, words_idx, new_matrix, top)

print ("Standardowe poszukiwanie:", find_similar_with_rejection("mateusza",[] , words_idx, embeddings))
print ("Poszukiwanie po projekcji:", find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings))

Standardowe poszukiwanie: ['łukasza', 'ewangelii', 'ewangelisty', 'ewangelia', 'bartłomieja', 'ewangeliach', 'apostoła', 'mateusz', 'tymoteusza', 'jakuba']
Poszukiwanie po projekcji: ['macieja', 'andrzeja', 'antoniego', 'stanisława', 'marcina', 'michała', 'kacpra', 'bartłomieja', 'rafała', 'sebastiana']


In [73]:
assert "ewangelii" in find_similar_with_rejection("mateusza",[] , words_idx, embeddings)
assert "ewangelii" not in find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings)
assert "ewangelisty" not in find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings)


Analogicznie słowo ,,java'' jest nie tylko nazwą języka programownia (https://pl.wikipedia.org/wiki/Java_(ujednoznacznienie)) -- jest np. nazwą geograficzną (indonezyjska wyspa koło Sumatry). Sprawdź jakie wyrazy są podobne do "java" oraz po odrzuceniu kierunku "javascript" (tj. kierunku związanego z językami programowania).

In [74]:
# YOUR CODE HERE
print(find_similar_with_rejection('java', None, words_idx, embeddings))
print(find_similar_with_rejection('java', ['javascript'], words_idx, embeddings))

['javascript', 'javy', 'c#', 'c++', 'programowania', 'implementacja', 'lisp', 'framework', 'api', 'programistyczne']
['tromp', 'sumatra', 'niszczycielami', 'indonezja', 'penang', 'fregatą', 'jawa', 'lion', 'krążowniki', 'niszczyciele']


Spróbuj poekseprymentować samemu!

In [78]:
# YOUR CODE HERE
print(find_similar_with_rejection('zamek', None, words_idx, embeddings))
print(find_similar_with_rejection('zamek', ['klucz'], words_idx, embeddings))
print(find_similar_with_rejection('zamek', ['budynek'], words_idx, embeddings))

['zamku', 'zamkiem', 'zamki', 'zamkowy', '„zamek', 'zamków', 'zamkowe', 'zameczek', 'pałac', 'warownia']
['zamku', 'zamkiem', 'zamkowe', 'zamkowym', 'castle', 'zamkowy', '„zamek', 'zamków', 'warownia', 'warowni']
['zamki', 'zamkiem', 'zamku', 'zamków', 'zamkach', 'zamkami', '„zamek', 'zamka', 'castle', 'zamkowe']


Wykonanie projekcji w przestrzeni zagnieżdżeń może być jedną z prostych technik zwalczenia tzw. gender bias (http://wordbias.umiacs.umd.edu/) w reprezentacji słów. Okazuje się, że wykonanie projekcji macierzy zagnieżdżeń na przestrzeń w której ,,brakuje kierunku he-she'' może być bardzo prostą techniką zredukowania tego typu obciążenia.

## Zadanie 2 - zagnieżdżenia dokumentów
W tym ćwiczeniu powócimy do zbioru tweetów, który analizowaliśmy w poprzednim dokumencie.

In [79]:
from helpers import DataSet
training_set = DataSet(['tweets.txt'])

Reading data set ['tweets.txt']


In [80]:
for i in training_set.tweets:
    print(i.text)
    print(i.tokens)
    print(i.clazz)
    break

dear @Microsoft the newOoffice for Mac is great and all, but no Lync update? C'mon.
['dear', '@microsoft', 'the', 'newooffice', 'for', 'mac', 'is', 'great', 'and', 'all', ',', 'but', 'no', 'lync', 'update', '?', "c'mon", '.']
negative


Tym razem do zbudowania reprezentacji będziemy używać narzędzie Universal Sentence Encoder stworzone przez Googla na bazie głębokiej sieci uśredniającej (i architektur rekurencyjnych). Poniższy kod pokazuje sposób użycia tego narzędzia. 
Kod spokojnie można wywoływać na CPU -- choć ściąganie modelu trochę może potrwać.

In [82]:
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding"])
print (embeddings)

tf.Tensor(
[[-0.03133018 -0.06338634 -0.01607502 ... -0.03242778 -0.04575741
   0.05370456]
 [ 0.05080861 -0.01652428  0.01573781 ...  0.00976659  0.03170123
   0.0178812 ]], shape=(2, 512), dtype=float32)


Wykorzystując reprezetnację USE wytrenuj wybrany klasyfikator z pakietu `sklearn` i zweryfikuj jego jakość działania.

In [83]:
# YOUR CODE HERE
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

texts = list(embed([tweet.text for tweet in training_set.tweets]))
classes = [tweet.clazz for tweet in training_set.tweets]
x_train, x_test, y_train, y_test = train_test_split(texts, classes, random_state=8348)

classifier = SVC()
classifier.fit(x_train, y_train)
accuracy_score(y_test, classifier.predict(x_test))

0.6534798534798535

Skomentuj wyniki i odnieś je do wyników z poprzedniego zadania domowego. Na ile użycie reprezentacji rozproszonych pozwoliło na poprawę wyników?

In [84]:
# YOUR CODE HERE
# Reprezentacje rozproszone: 0.6534798534798535
# Hashowanie: 0.5100794135613927
# BOC: 0.49114233353695785

# Zadanie 3 - konstruowanie zagnieżdżeń
W tym ćwiczeniu kontynuujemy pracę z tweetami, ale pomijamy całkowicie ich klasy. Zbiór tweetów potraktujemy jako korpus do nauczenia zagnieżdżeń słów przy pomocy macierzy PMI.
- Wypełnij macierz kontekst - dokument przy użyciu symetrycznego okna o promieniu 4 (po 4 słowa w każdą stronę)
- Możesz ograniczyć słownictwo do 10K słów
- Przekształć macierz w macierz PPMI
- Stwórz zagnieżdżenia wykorzystując dekompozycję SVD do wybranej wymiarowości $d$ (ze względu na koszt obliczeniowy może to być mała wymiarowość np. $d=10$)

In [101]:
# YOUR CODE HERE
from sklearn.decomposition import TruncatedSVD
from collections import Counter
from numpy.linalg import svd
from math import log2

r = 4 # radius
v = 10000 # vocabulary size
d = 10 # dimensions

def train():
    
    ## VOCABULARY
    
    words = Counter()
    for row in training_set.tweets:
        words.update(row.tokens)
        
    voc = {}
    for index, (word, _) in enumerate(words.most_common(v)):
        voc[word] = index
    
    ## HAL
    
    context = np.zeros((2*v, 2*v))
    for row in training_set.tweets:
        for left, word, right in window_gen(row.tokens):
            if word in voc:
                neighbours = left + right
                for neighbour in neighbours:
                    if neighbour in voc:
                        context[voc[word], voc[neighbour]] += 1
            
    ## PPMI
    
    ppmi_matrix = context.copy()
    (row_count, col_count) = ppmi_matrix.shape
    total_sum = np.sum(ppmi_matrix)
    row = np.sum(ppmi_matrix, axis=1) / total_sum
    col = np.sum(ppmi_matrix, axis=0) / total_sum
    ppmi_matrix = ppmi_matrix / total_sum
    
    for r_i in range(0, row_count):
        
        for c_i in range(0, col_count):    
            row_times_col = row[r_i] * col[c_i]
            
            if row_times_col == 0:
                ppmi_matrix[r_i, c_i] = 0
            else:
                pmi_value = ppmi_matrix[r_i, c_i] / row_times_col
                
                if pmi_value == 0:
                    ppmi_matrix[r_i, c_i] = 0
                else:
                    ppmi_matrix[r_i, c_i] = max(0, log2(pmi_value))
                
    ## SVD
                
    svd = TruncatedSVD(d)
    result = svd.fit_transform(ppmi_matrix)
    return result, voc


def window_gen(row):
    width = len(row)
    for offset in range(-r, width - r):
        left = max(offset, 0)
        mid = offset + r
        right = max(offset + 2 * r + 1, 0)
        
        item_left = row[left : mid]
        item = row[mid]
        item_right = row[mid + 1 : right]
        
        yield item_left, item, item_right


embeddings, words = train()

Przetestuj działanie Twoich zagnieżdżeń wykorzystując funkcję `find_similar` na wybranych słowach.

In [102]:
# YOUR CODE HERE
find_similar('xbox', words, embeddings)

['ouija',
 'shop',
 'today-',
 'paralytic',
 'overhyped',
 'garage',
 'massive',
 'supposed',
 'http://t.co/q3fenbq151',
 'seriously']